# Add Environmental Noises to The Audio ( speech )

# Speech collected from the site - https://openslr.org/83/
# Noise collected from the site - https://www.freesoundeffects.com/

* Speech and Noise must be sampled at same rate - 16K ( make your own choice )
* Noise samples must be greater than the speech sample so that 
* Maracas Library works with above two conditions (Same sampling rate & [len(noise) > len(speech)]  :   https://pypi.org/project/maracas/

In [1]:
from maracas.dataset import Dataset
import numpy as np

In [2]:
speechPathA = './AWAV'
speechPathB = './BWAV'
NoisesA = './noisesA'
NoisesB = './noisesB'
NoiseSpeechWavB = './BWAVNOISE'

In [3]:
import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')
from scipy.io import wavfile
import scipy.signal as sps
import math

# Down Sample the sampling rate to 16K

* Make folder A for speech data
* Down sample and dump speech with 16K sampled to folder B
* Copy data in folder A and folder B initially with same wav files, downsample will replace files in folder B

In [27]:
A = os.path.join(speechPathA)
B = os.path.join(speechPathB)

img_list = os.listdir(A)
num_wavs = len(img_list)
new_rate = 16000

for n in range(num_wavs):
    name_A = img_list[n]
    name_B = name_A
    path_A = os.path.join(A,name_A)
    path_B = os.path.join(B,name_B)
    print(path_A, path_B)
    if os.path.isfile(path_A) and os.path.isfile(path_B):
        sampling_rate, data = wavfile.read(path_A)
#         print(sampling_rate)
         # Resample data
        number_of_samples = round(len(data) * float(new_rate) / sampling_rate)
        data = sps.resample(data, number_of_samples)
        data = np.asarray(data, dtype=np.int16)
        wavfile.write(path_B,new_rate,data)
#         sampling_rate, data = wavfile.read(path_B)
#         print(sampling_rate)
    else:
        print("error")

# Resample noises to 16K

In [26]:
A = os.path.join(NoisesA)
B = os.path.join(NoisesB)

img_list = os.listdir(A)
num_wavs = len(img_list)
new_rate = 16000

print(num_wavs)

for n in range(num_wavs):
    name_A = img_list[n]
    name_B = name_A
    path_A = os.path.join(A,name_A)
    path_B = os.path.join(B,name_B)
    print(path_A, path_B)
    if os.path.isfile(path_A) and os.path.isfile(path_B):
        sampling_rate, data = wavfile.read(path_A)
#         print(sampling_rate)
         # Resample data
        number_of_samples = round(len(data) * float(new_rate) / sampling_rate)
        data = sps.resample(data, number_of_samples)
        data = np.asarray(data, dtype=np.int16)
        wavfile.write(path_B,new_rate,data)
#         sampling_rate, data = wavfile.read(path_B)
#         print(sampling_rate)
    else:
        print("error")

## Maintain sampling rate = 16K
## Make All noise files longer than speech file##
#### Find maximum length of the speech file in the datasets and keep length of the noise file more than maximum lenght of the speech file ( 50 sec added to noise )

In [25]:
A = os.path.join(NoisesA)
B = os.path.join(NoisesB)

img_list = os.listdir(A)
num_wavs = len(img_list)
new_rate = 16000


for n in range(num_wavs):
    name_A = img_list[n]
    name_B = name_A
    path_A = os.path.join(A,name_A)
    path_B = os.path.join(B,name_B)
    print(path_A, path_B)
    if os.path.isfile(path_A) and os.path.isfile(path_B):
        sampling_rate, data = wavfile.read(path_A)
        length_s = len(data)/float(sampling_rate)
        if(length_s < 50.0):
            n = math.ceil(50*sampling_rate/len(data))
        data = np.tile(data,n)
        
        number_of_samples = round(len(data) * float(new_rate) / sampling_rate)
        data = sps.resample(data, number_of_samples)
        data = np.asarray(data, dtype=np.int16)
        wavfile.write(path_B,new_rate,data)

# Add Speech ( 16 K Sampled )

In [8]:
# Make sure this is reproducible
np.random.seed(42)

d = Dataset()

# All files can be added one by one or by folder. Adding a folder will add
# all speech files inside that folder recursively if recursive=True.
d.add_speech_files(speechPathB, recursive=True)

# Add Noise with sampling rate = 16K & length greater than the speech

In [9]:
# When adding noises, you can give a "nickname" to each noise file. If you do not
# give it a name, the name will be the file name without the '.wav' extension
d.add_noise_files(NoisesB)

# Add maximum one noises in the speech under 5 different DB conditions

In [14]:
# NoiseSpeechWavB = './BWAVNOISE'

In [24]:
# When generating a dataset, you can choose which SNRs will be used and how many
# files per condition you want to be generated. 
for i in range(10): # run below lines multiple time
    d.generate_dataset([-6, -3, 0, 3, 6], NoiseSpeechWavB, files_per_condition = 1)

### Copy Speech + Noise File

In [19]:
import os
import shutil

In [20]:
# copy noisy file to './BWAN' from NoiseSpeechWavB = './BWAVNOISE'

In [21]:
DEST =  './BWAVN'
DESTPATH = os.path.join(DEST)

for root, dirs, files in os.walk("./BWAVNOISE"):
    for file in files:
        if file.endswith(".wav"):
#             print(file)
            shutil.copy(os.path.join(root, file), DEST)

### Target to make WAVA and WAVB folder for speech & speech+noise for same wav files
* WAVA consists [clean speech] and WAVB consists of [speech+noise] for same wav files
* Copy noisy wav files from BWAVN to WAVB
* Check noise file names in WAVB and copy clean wav files matching to noise in WAVA

In [22]:
# Copy noisy wav files from BWAVN to WAVB

source = './BWAVN'
destination = './WAVB'

for filename in glob.glob(os.path.join(source, '*.wav')):
    shutil.copy(filename, destination)

In [23]:
# BWAV - find files which are present in WAVB(BWAVN)
# WAVA, WAVB

pathWavA = './WAVA'
pathWavB = './WAVB'
pathWavC =  './BWAV' # Original speech file sampled at 16K


A = os.path.join(pathWavA)
B = os.path.join(pathWavB)
C = os.path.join(pathWavC)


wav_list_C = os.listdir(C)
num_wavs_C = len(wav_list_C)

wav_list_B = os.listdir(B)
num_wavs_B = len(wav_list_B)

for m in range(num_wavs_C):
    name_C = wav_list_C[m]
    path_C = os.path.join(C,name_C)
    if name_C.endswith(".wav"):
        for n in range(num_wavs_B):
            name_B = wav_list_B[n]
            if name_B.endswith(".wav"):
                if name_C == name_B :
                    shutil.copy(path_C, pathWavA)
                    break